In [ ]:
!pip install azure-ai-textanalytics==5.2.0b1
!pip install bs4
!pip install requests

In [6]:
# authenticate azure text analytics client
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

key = "677e7a68d2f94932a97572bfd9505ea3"
endpoint = "https://covid-corruption-nlp2.cognitiveservices.azure.com/"

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()


In [ ]:
# perform key phrase extraction
def key_phrase_extraction(client, documents=""):
    try:
        responses = client.extract_key_phrases(documents = documents)
        
        key_phrases = []
        for response in responses:
            if response.key_phrases:
                key_phrases += response.key_phrases
        
        return key_phrases

    except Exception as err:
        print("Encountered exception. {}".format(err))


news_source = "sky"
f_in = open('datasets/parsed-news/{}.txt'.format(news_source), 'r')
documents = [line.rstrip() for line in f_in.readlines()]
num_docs = len(documents)

key_phrases = []
for i in range(0, num_docs, 10):
    print(i)
    key_phrase = key_phrase_extraction(
                    client, 
                    documents=documents[i: min(num_docs, i+10)]
    )

    if key_phrase:
        key_phrases += key_phrase

print(key_phrases)
f_out = open('datasets/parsed-news/{}_phrases.txt'.format(news_source), 'w+')
for phrase in key_phrases:
    phrase = '-'.join(phrase.split())
    f_out.write(phrase + '\n')

In [3]:
# opinion mining on trustpilot reviews
from bs4 import BeautifulSoup
import requests
import pandas as pd

grouped_details = pd.read_csv('datasets/grouped_details.csv')
link_cols = [x for x in grouped_details.columns if 'company_link' in x]

def get_reviews(x):
    results = []
    for cell in x:
        if pd.isna(cell) or not cell:
            results.append('')
        else:
            page = requests.get("https://uk.trustpilot.com/review/"+cell)
            print(page)
            soup = BeautifulSoup(page.content, "html.parser")
            reviewcontent = soup.find_all('p', {'class', 'review-content__text'})
            
            results.append([review.get_text() for review in reviewcontent if review])
    return results

link_details = pd.DataFrame(grouped_details[link_cols])

In [4]:
# tr_reviews = link_details.apply(get_reviews)
# tr_reviews.to_csv('reviews.csv')

In [7]:
def opinion_mining(client,documents):

    result = client.analyze_sentiment(documents, show_opinion_mining=True)
    doc_result = [doc for doc in result if not doc.is_error]

    negative_keywords=[]
    for document in doc_result:
        for sentence in document.sentences:
            for mined_opinion in sentence.mined_opinions:
                if mined_opinion.target.sentiment == 'negative':
                    negative_keywords.append(mined_opinion.assessments[0].text)
                else:
                    negative_keywords.append('')
    return negative_keywords 

tr_reviews =  pd.read_csv('reviews.csv')
keywords = []
for review in tr_reviews['company_link_0']:
    #print(type(review))
    if type(review) == str: 
        review = eval(review)
        num_docs = len(review)
        for i in range(0, num_docs, 10):
            keywords.append(opinion_mining(client,documents=review[i: min(num_docs, i+10)]))

In [8]:
keywords = pd.DataFrame(keywords)
keywords['company_name'] = grouped_details['company_name']
keywords.to_csv('datasets/tp_keywords.csv')